In [1]:
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model
import pathlib
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import glob
from keras.preprocessing import image
import numpy as np
from keras.models import load_model

In [2]:
base_model = tf.keras.applications.ResNet50(input_shape=(224, 224, 3), include_top=True, weights="imagenet", input_tensor=None, pooling='max', classes=1000,)
base_output = base_model.output
final_output = Dense(2, activation='softmax')(base_output)

for layer in base_model.layers:
    layer.trainable = False


102973440/102967424 [==============================] - 5s 0us/step


In [3]:
def make_generator(dataset):
    dataset_path = pathlib.Path('data', dataset)

    datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
                                 
    return datagen.flow_from_directory(str(dataset_path),
                                       target_size=(224, 224),
                                       batch_size=32,
                                       shuffle=True,
                                       class_mode='categorical')

In [5]:
train_generator = make_generator('train')
validation_generator = make_generator('test')

model = Model(inputs=base_model.input, outputs=final_output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_generator,
    epochs=5,
    steps_per_epoch=300,
    validation_data=validation_generator,
    validation_steps=16)

model.save('iscute.h5')

Found 10143 images belonging to 2 classes.
Found 1509 images belonging to 2 classes.
Epoch 1/5
300/300 [==============================] - 1054s 4s/step - loss: 0.6906 - accuracy: 0.5678 - val_loss: 0.6884 - val_accuracy: 0.6016
Epoch 2/5
300/300 [==============================] - 805s 3s/step - loss: 0.6850 - accuracy: 0.6073 - val_loss: 0.6831 - val_accuracy: 0.6074
Epoch 3/5
300/300 [==============================] - 253s 843ms/step - loss: 0.6806 - accuracy: 0.6069 - val_loss: 0.6775 - val_accuracy: 0.6426
Epoch 4/5
300/300 [==============================] - 253s 844ms/step - loss: 0.6774 - accuracy: 0.6099 - val_loss: 0.6746 - val_accuracy: 0.5938
Epoch 5/5
300/300 [==============================] - 253s 845ms/step - loss: 0.6751 - accuracy: 0.6015 - val_loss: 0.6692 - val_accuracy: 0.6562


In [6]:
model.fit(train_generator,
    epochs=10,
    steps_per_epoch=300,
    validation_data=validation_generator,
    validation_steps=16)

model.save('iscute2.h5')

Epoch 1/10
300/300 [==============================] - 252s 840ms/step - loss: 0.6719 - accuracy: 0.6063 - val_loss: 0.6748 - val_accuracy: 0.5879
Epoch 2/10
300/300 [==============================] - 251s 837ms/step - loss: 0.6695 - accuracy: 0.6126 - val_loss: 0.6724 - val_accuracy: 0.6035
Epoch 3/10
300/300 [==============================] - 252s 840ms/step - loss: 0.6668 - accuracy: 0.6121 - val_loss: 0.6633 - val_accuracy: 0.5996
Epoch 4/10
300/300 [==============================] - 252s 839ms/step - loss: 0.6681 - accuracy: 0.6063 - val_loss: 0.6721 - val_accuracy: 0.6016
Epoch 5/10
300/300 [==============================] - 253s 843ms/step - loss: 0.6640 - accuracy: 0.6138 - val_loss: 0.6548 - val_accuracy: 0.6406
Epoch 6/10
300/300 [==============================] - 252s 840ms/step - loss: 0.6645 - accuracy: 0.6128 - val_loss: 0.6676 - val_accuracy: 0.6113
Epoch 7/10
300/300 [==============================] - 252s 839ms/step - loss: 0.6630 - accuracy: 0.6160 - val_loss: 0.6471 -

In [7]:
def predict(model_loaded, filename):
    img = image.load_img(filename, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    return model_loaded.predict (images)

In [8]:
model_loaded = load_model('iscute2.h5')
model_loaded.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

correct = 0
falsePositives = 0
falseNegatives = 0
total = 0
print('not cute, cute')
for filename in glob.glob('data/predict/*'):
    classes = predict(model_loaded, filename)
    if (len(filename) == 33):
        total += 1
        if (classes[0][0] > classes[0][1]):
            correct += 1
            print('correct')
        else:
            falsePositives += 1
    elif(len(filename) == 24):
        total += 1
        if (classes[0][0] < classes[0][1]):
            correct += 1
            print('correct')
        else:
            falseNegatives += 1
    print(filename + ': ' + str(classes))

print('not cute, cute')
print('correct: ' + str(correct) + ' (' + str(correct / total) + '%)')
print('FP: ' + str(falsePositives) + ', FN: ' + str(falseNegatives))

not cute, cute
data/predict\000ada55d36b4bcb.jpg: [[0.15954116 0.84045887]]
data/predict\00a300e8b0cef4d3.jpg: [[0.16837561 0.83162445]]
correct
data/predict\00a3654c1cf00d11.jpg: [[0.5946191  0.40538093]]
data/predict\00a36f96e31731c4.jpg: [[0.16574934 0.8342506 ]]
data/predict\00a42a80e5e8d194.jpg: [[0.21026315 0.78973687]]
data/predict\00a7655d4eabf186.jpg: [[0.1970007 0.8029993]]
data/predict\00abfe9035972732.jpg: [[0.23064129 0.7693587 ]]
data/predict\00ac70478d4727bf.jpg: [[0.11923692 0.8807631 ]]
correct
data/predict\00acf53b127218c2.jpg: [[0.59964865 0.40035135]]
correct
data/predict\01IrQfV.jpg: [[0.30891746 0.6910825 ]]
data/predict\01KyAvz.jpg: [[0.67662096 0.323379  ]]
correct
data/predict\01ZSRgh.jpg: [[0.23858742 0.76141256]]
correct
data/predict\0thB4ca.jpg: [[0.20742321 0.7925768 ]]
correct
data/predict\0TizTVN.jpg: [[0.32586297 0.67413706]]
correct
data/predict\0tOXueK.jpg: [[0.42276764 0.57723236]]
correct
data/predict\0TskMIN.jpg: [[0.23823695 0.76176304]]
correct
da